In [4]:
pip install textattack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.8/436.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 52.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 57.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 21.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 k

In [5]:
import json
import string
import random
from textattack.augmentation import WordNetAugmenter

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
def read_json(path):
    with open(path, 'r') as json_file:
        data = json.load(json_file)
    return data
def write_json(path, data):
    with open(path, 'w') as file:
        # Write the Python data structure as JSON to the file
        json.dump(data, file, indent=2)

In [7]:
config_path = "/content/drive/MyDrive/NAACL/model/config.json"
config = read_json(config_path)
left_padding = config["left_padding"]
right_padding = config["right_padding"]

punctuation_error = [
        "%"
    ]
wordnet_aug = WordNetAugmenter()

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [8]:
def remove_punctuation(text):
    # Create a string of all punctuation characters
    punctuation_chars = string.punctuation

    # Remove punctuation at the beginning and end of the string
    cleaned_text = text.strip(punctuation_chars).strip().strip(punctuation_chars).strip().strip(punctuation_chars).strip()

    return cleaned_text
def span(text, sub_text, s_ind=0):
    sub_text = remove_punctuation(sub_text)
    new_text = text.split()
    if text == sub_text:
        ss = s_ind
        se = s_ind+len(new_text)
        return ss, se, text
    new_sub_text = sub_text
    new_sub_text = new_sub_text.split()
    check = True
    for i in range(len(new_text)):
        temp = i
        for j in range(len(new_sub_text)):
            if new_sub_text[j] != new_text[i + j]:
                if j == len(new_sub_text)-1:
                    for punctuation in punctuation_error:

                        if new_sub_text[j]+punctuation == new_text[i + j]:
                            # print(new_sub_text)
                            new_sub_text[j] = new_sub_text[j]+punctuation
                            ss = i
                            se = i+j
                            print(new_sub_text)
                            return s_ind+ss, s_ind+se+1, ' '.join(new_sub_text)
                check = False
                break
            else:
                temp = i + j
                check = True
        if check == True:
            ss = i
            se = temp
            return s_ind+ss, s_ind+se+1, sub_text

def convertraw_to_train(data):
    new_data = []
    max_len = 0
    for i in range(len(data)):
        conversation_ID = data[i]['conversation_ID']
        conversation = data[i]['conversation']

        utterance_ID_pool = []
        text_pool = []
        speaker_pool = []
        emotion_pool = {}
        for j in range(len(conversation)):
            utterance_ID_pool.append(conversation[j]['utterance_ID'])
            text_pool.append(conversation[j]['text'])
            if max_len < len(conversation[j]['text'].split()):
                max_len = len(conversation[j]['text'].split())
            speaker_pool.append(conversation[j]['speaker'])
            emotion_pool[j+1] = {}
            emotion_pool[j+1]["property"] = {}
            emotion_pool[j+1]["casual"] = {}
            if "emotion" in conversation[j].keys():
                emotion_pool[j+1]["property"] = conversation[j]['emotion']


        if "emotion-cause_pairs" in data[i].keys():
            cause_pairs_pool = []
            cause_pairs = data[i]['emotion-cause_pairs']
            for j in range(len(cause_pairs)):
                emotion_utter = cause_pairs[j][0]
                casual_utter = cause_pairs[j][1]
                utter, emotion = emotion_utter.split("_")
                casual_utter, casual_text = casual_utter.split("_")
                # if conversation_ID == 1049:
                #     print("cause_pairs[j][0]:" +str(cause_pairs[j][0]))
                #     print("cause_pairs[j][1]:" +str(cause_pairs[j][1]))
                #     print("utter:" +str(utter))
                #     print("casual_utter:" +str(casual_utter))
                #     print("int(utter) - left_padding :" +str(int(utter) - left_padding))
                #     print("int(utter) + right_padding :" +str(int(utter) + right_padding))
                if int(utter) - left_padding > int(casual_utter):
                    continue
                if int(utter) + right_padding < int(casual_utter):
                    continue
                start, end, _ = span(text_pool[int(casual_utter)-1], casual_text)
                emotion_pool[int(utter)]["casual"][int(casual_utter)] = {
                    "casual_text": casual_text,
                    "start": start,
                    "end": end
                }


        for j in range(len(conversation)):
            start_ids = max(1, j-left_padding+1)
            end_ids = min(j+right_padding+2, len(conversation)+1)
            for k in range(start_ids, end_ids):
                if k not in emotion_pool[j+1]["casual"].keys():
                    emotion_pool[j+1]["casual"][k] = {
                        "casual_text": "",
                        "start": 0,
                        "end": 0
                    }

            emotion_pool[j+1]["casual"] = dict(sorted(emotion_pool[j+1]["casual"].items()))

            pool = []
            for k in emotion_pool[j+1]["casual"].keys():
                pool.append({
                    "utter_ID": k,
                    "casual_text": emotion_pool[j+1]["casual"][k]["casual_text"],
                    "start": emotion_pool[j+1]["casual"][k]["start"],
                    "end": emotion_pool[j+1]["casual"][k]["end"]
                })
            emotion_pool[j+1]["casual"] = pool

            start_ids = max(0, j-left_padding)
            end_ids = min(j+right_padding+1, len(conversation))
            paragraph = " ".join(text_pool[start_ids: end_ids])

            new_data.append({
                "conversation_ID" : conversation_ID,
                "utterance_ID" : utterance_ID_pool[j],
                "paragraph": paragraph,
                "emotion": emotion_pool[j+1],
                "text_pool": text_pool[start_ids: end_ids],
                "speaker_pool": speaker_pool[start_ids: end_ids],
            })
    print(max_len)
    return new_data

def count(data):
    dictionary = {}

    for item in data:
        # print( dictionary.keys())
        emotion = item['emotion']['property']
        if emotion in dictionary.keys():
            # print(emotion)
            dictionary[emotion] +=1
        else:
            # print(emotion)
            dictionary[emotion] =1
    return dictionary

def count_dict(data):
    dictionary = {}

    for item in data.keys():
        dictionary[item] = len(data[item])
    return dictionary

def count_dis(data):
    dict_dis = {}
    num = 0
    for i in range(len(data)):
        emotion_casual_pairs = data[i]['emotion-cause_pairs']
        for j in range(len(emotion_casual_pairs)):
            emotion_utter = emotion_casual_pairs[j][0]
            casual_utter = emotion_casual_pairs[j][1]
            utter, emotion = emotion_utter.split("_")
            casual_utter, casual_text = casual_utter.split("_")
            utter = int(utter)
            casual_utter = int(casual_utter)
            num += 1
            if utter - casual_utter not in dict_dis.keys():
                dict_dis[utter - casual_utter] = 1
            else:
                dict_dis[utter - casual_utter] += 1

    dict_dis = dict(sorted(dict_dis.items(), key=lambda item: item[1]))
    dict_r = {}
    for j in dict_dis.keys():
        dict_r[j] = dict_dis[j]/num

    return dict_dis, dict_r
def augment_data(data):
    dict = {}
    for item in data:
        property = item['emotion']['property']
        if property not in dict:
            dict[property] = [item]
        else:
            dict[property].append(item)

    num_elements_to_choose = len(dict['neutral']) // 3

    # Randomly choose one-third of the elements
    dict['neutral'] = random.sample(dict['neutral'], num_elements_to_choose)

    count_appear = count_dict(dict)
    max_appear = max(count_appear.values())

    num_augment = {}
    for emotion in count_appear.keys():
        num_augment[emotion] = max_appear - count_appear[emotion]
    print("num_augment : "+str(num_augment))

    augment_dict = {
        "joy": [],
        "neutral": []
    }

    for emotion in dict.keys():
        for i in range(num_augment[emotion]):
            index = i % len(dict[emotion])
            sample = augment_sample(dict[emotion][index])
            if emotion not in augment_dict:
                augment_dict[emotion]=[sample]
            else:
                augment_dict[emotion].append(sample)
            # print(dict[emotion][index])
            # print(sample)
            # print("====")

    train_dict = {}
    test_dict = {}
    train_data = []
    test_data = []
    print(dict.keys())
    print(augment_dict.keys())
    for emotion in dict.keys():
        print(emotion)
        list = dict[emotion] + augment_dict[emotion]
        n_train = int(0.8*len(list))
        train_dict[emotion] = list[:n_train]
        test_dict[emotion] = list[n_train:]
        train_data.extend(train_dict[emotion])
        test_data.extend(test_dict[emotion])
    # new_data = neutral_data[0:int(len(neutral_data)/3)] + other_data



    random.shuffle(train_data)
    random.shuffle(test_data)
    return train_data, test_data
def augment_sample(sample):
    emotion = sample["emotion"]
    text_pool = sample["text_pool"]

    for i in range(len(text_pool)):
        if emotion['casual'][i]["casual_text"] == "":
            text_pool[i] = augment_text(text_pool[i])[0]
            # print(augment_text(text_pool[i]))

    paragraph = ""
    for i in range(len(text_pool)):
        # print(new_text_pool[i])
        paragraph += " " + text_pool[i]

    new_sample = {
        'conversation_ID': sample['conversation_ID'],
        'utterance_ID': sample['utterance_ID'],
        'paragraph': paragraph,
        'emotion':
        {
            'property': sample['emotion']['property'],
            'casual': sample['emotion']['casual']
        },
        'text_pool': text_pool,
        'speaker_pool': sample['speaker_pool']
    }
    return new_sample
def augment_text(text):
    return wordnet_aug.augment(text)

In [9]:
data = read_json("/content/drive/MyDrive/HaruLab/Dataset/ECF 2.0/train/Subtask_1_train.json")
# count_dis(data)
new_data= convertraw_to_train(data)
# write_json("../data/ECF 2.0/train/train.json", new_data)

78


In [10]:
train_data, test_data = augment_data(new_data)
write_json("/content/drive/MyDrive/HaruLab/Dataset/ECF 2.0/train/augment_train.json", train_data)
write_json("/content/drive/MyDrive/HaruLab/Dataset/ECF 2.0/train/augment_valid.json", test_data)

num_augment : {'neutral': 325, 'surprise': 461, 'anger': 686, 'sadness': 1154, 'joy': 0, 'disgust': 1887, 'fear': 1928}
dict_keys(['neutral', 'surprise', 'anger', 'sadness', 'joy', 'disgust', 'fear'])
dict_keys(['joy', 'neutral', 'surprise', 'anger', 'sadness', 'disgust', 'fear'])
neutral
surprise
anger
sadness
joy
disgust
fear


In [11]:
count(train_data)

{'neutral': 1840,
 'disgust': 1840,
 'fear': 1840,
 'joy': 1840,
 'anger': 1840,
 'surprise': 1840,
 'sadness': 1840}

In [12]:
count(test_data)

{'joy': 461,
 'neutral': 461,
 'surprise': 461,
 'sadness': 461,
 'fear': 461,
 'disgust': 461,
 'anger': 461}

In [13]:
data = read_json("/content/drive/MyDrive/HaruLab/Dataset/ECF 2.0/test/Subtask_1_test.json")
new_data = convertraw_to_train(data)
write_json("/content/drive/MyDrive/HaruLab/Dataset/ECF 2.0/test/test.json", new_data)

63
